In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

In [2]:
%run beautify_plots.py

In [3]:
# My standard magic !  You will see this in almost all my notebooks.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

%matplotlib inline

In [4]:
import pandas as pd
import numpy as plt

import matplotlib.pyplot as plt

import os 


# Convolutional Neural Networks: the spatial dimensions

Our treatment, thus far, of Neural Networks has been rather limited.  An example has consisted of
- Multiple features
- At a single spatial location
- Represented as a vector of shape $(1 \times n_\llp)$
    - But we often ignored the singleton dimension

But the natural world's spatial dimensions are much higher than 1 !
- $N > 1$ dimensions
- Our examples become $(N +1)$ dimensional
- Represented as a vector of shape $(\dim_{\llp,1} \times \dim_{\llp,2} \times \ldots \dim_{\llp,N} \,\, \times n_\llp )$


When $N = 1$ and $\dim_1 =1$
- we have our case of $n_\llp$ features at a single location

We have shown that permuting the order of features has no effect on a Dense layer
- There is no ordering relationship among features

But when $\dim_1 > 1$, there is a *spatial ordering*.  For example
- a 2D image
- time ordered data

We need some terminology to distinguish the final dimension from the non-final dimensions

Suppose $\y_\llp$ is $(N_\llp+1)$ dimensional of shape 
$$
|| \y_{\llp} || = (\dim_{\llp,1} \times \dim_{\llp,2} \times \ldots \dim_{\llp,N_\llp} \,\, \times n_{\llp} )
$$

(Thus far: $N_\llp = 1$ and $n_{\llp} = 1$ but that will soon change)

The first $N_\llp$ dimensions $(\dim_{\llp,1} \times \dim_{\llp,2} \times \ldots \dim_{\llp,N} )$
- Are called the *spatial* dimensions of layer $\ll$

The last dimension (of size $ n_{\llp}$)
- Indexes the  features i.e., varies over the number of features
- Called the *feature* or *channel* dimension


**Notation**

- $N_\llp$ denotes the *number* of spatial dimensions of layer $\ll$
- $n_\llp$ denotes the *number of features* in layer $\ll$
- We elide the spatial dimensions as necessary, writing 
$$\y_{\llp, \ldots, j}
$$
to denote *feature map* $j$ of layer $\ll$
    - where the dots ($\ldots$) indicate the $N_\llp$ spatial dimensions
    - e.g., the feature map detecting a "smile" in the image of a face

For example
- A grey-scale image
    - $N = 2, n_\llp = 1$
    - Each pixel in the image has one feature
        - the grey-scale intensity 
    - There is an ordering relationship between 2 pixels
        - "left/right", "above/below"
- A color image
    - $N = 2, n_\llp = 3$ 
    - Each pixel in the image has 3 features/attributes
        - the intensity of each of the colors

One can imagine even higher dimensional data ($N > 2$)
- Equity data with "spatial location" identified by (Month, Day, Time)
    - With attributes: $\{ $ Open, High, Low, Close $\}$
    - Month/Day/Time are ordered

Note the distinction between the cases
- When layer $\ll$ has dimension $(\dim_\llp \times 1)$
    - a single feature
    - at $\dim_\llp = \dim_{(\ll-1)}$  *spatial* locations
- When layer $\ll$ has dimension $(1 \times \dim_\llp)$
    - (which is how we have implicitly been considering vectors when discussing the Dense layer type)
    - $\dim_\llp = \dim_{(\ll-1)}$ features
    - at a single spatial location

$n_\llp$ will always refer to the *number of features* of a layer $\ll$

Here is a [picture](CNN_pictorial.ipynb#Conv-1D:-single-feature) of a Convolutional layer $\ll$
transforming 
- a  1-dimensional input layer $(\ll-1)$  consisting of a single feature 
    - $N_{(\ll-1)} = 1, n_{(\ll-1)} =1$
- into a 1-dimensional output layer $\ll$  consisting of a single feature 
    - $N_\llp = 1, n_\llp =1$

We will generalize Convolution to deal with
- $N_\llp > 1$ spatial dimensions
- $n_\llp > 1$ features

As a preview of concepts to be introduced, consider
- the input layer $(l-1)$ is a two-dimensional ($N_{(\ll-1)} = 2$) grid of pixels
- $n_{(\ll-1)} = 1$
- layer $l$ is a Convolutional Layer identifying $n_\llp = 3$ features

<table>
    <tr>
        <th><center>Convolution: 1 input feature to 3 output features</center></th>
    </tr>
    <tr>
        <td><img src="images/Conv2d_multifeature_shape.png" width=80%></td>
    </tr>
</table>

Layer $(l-1)$ is three-dimensional tensor: $8 \times 8 \times 1$
- Spatial dimension $8 \times 8$
- 1 feature map (channel dimension $= 1$)

- Kernel $k_{\llp,j}$ is applied to each spatial location of layer $(\ll-1)$
- Detecting the presence of the pattern (defined by the kernel) at that location
    - kernel $k_{\llp,1}$ detects an eye
- Which results in feature map $\y_{\llp},\dots,j$ being created at layer $\ll$
    - $\y_{\llp,\dots,1}$ are indicators of the presence of an "eye" feature

**Convolutional Layer description**

With this terminology we can say that Convolutional Layer $\ll$:
- Transforms the $n_{(\ll-1)}$ feature maps of layer $(\ll-1)$
- Into $n_\llp$ feature maps of layer $\ll$
- Preserving the spatial dimensions: $\dim_{\llp,p} = \dim_{(\ll-1),p} \; 1 \le p \le N_{(\ll-1)}$
- Uses a different kernel $\kernel_{\llp,j}$ for each output feature/channel $1 \le j \le n_\llp$
- Applies this kernel to *each* element in the *spatial* dimensions
- Recognizing a single feature at each location within the spatial dimension


## Channel Last/First

We have adopted the convention of using the final dimension as the feature dimension.
- This is called *channel last* notation.

Alternatively: one could adopt a convention of the first channel being the feature dimension.
- This is called *channel first* notation.

When using a programming API: make sure you know which notation is the default
- Channel last is the default for TensorFlow, but other toolkits may use channel first.


# Conv1d transforming single feature to multiple features

Here is a [picture](CNN_pictorial.ipynb#Conv-1D:-single-feature-to-multiple-features) of a Convolutional layer $\ll$
transforming 
- a  1-dimensional input layer $(\ll-1)$  consisting of a single feature 
    - $N_{(\ll-1)} = 1, n_{(\ll-1)} =1$
- into a 1-dimensional output layer $\ll$  consisting of a *multiple* features 
    - $N_\llp = 1, n_\llp  > 1$
    


# Conv1d transforming multiple features to multiple features

What happens when the input layer has multiple features ?
- e.g., applying Convolutional layer $(\ll+1)$ to the $n_\llp$ features created by Convolutional layer $\ll$

The answer is 
- The kernels of layer $\ll$ also have a *feature* dimension
    - Kernel dimensions are $(f_\llp \times f_\llp \times n_{(\ll-1)})$
- This kernel is applied
    - at each spatial location
    - to *all features* of layer $(\ll-1)$
    - Computing a generalized "dot product": sum of element-wise products



<div>
    <br>
    <center><strong>Conv 1D: 2 input features: kernel 1</strong></center>
    <br>
<img src=images/Conv1d_2feature_kernel.png width="35%">
    <!-- edX: obsolete:: <img src="images/W9_L2_S23_Conv1d_2feature_kernel.png "> replace by EdX created image --> 
    <br>
    </div>

- $\mathbf{W}_{\llp, j', \ldots,j}$
    - layer $\ll$
    - output feature $j$
    - spatial location: $\ldots \in \{1,2,3\}$
    - input feature $j'$

Notice that (apart from combining spatial locations)
- multiple feature maps from layer $(\ll-1)$ are combined into one feature map at layer $\ll$.
- This is how the "left" half-smile and "right" half-smile features combine into the single "smile" feature

Here is a [picture](CNN_pictorial.ipynb#Conv-1D:-Multiple-features-to-multiple-features) of a Convolutional layer $\ll$
transforming 
- a  1-dimensional input layer $(\ll-1)$  consisting of a 2 features 
    - $N_{(\ll-1)} = 1, n_{(\ll-1)} = 2$
- into a 1-dimensional output layer $\ll$  consisting of a *multiple* features 
    - $N_\llp = 1, n_\llp  = 3$


With an input layer having $N$ spatial dimensions, a Convolutional Layer $\ll$ producing $n_\llp$ features
- Preserves the "spatial" dimensions of the input
- Replaces the channel/feature dimensions

That is\
$$
\begin{array}\\
|| \y_{(\ll-1)} || & = & (n_{(\ll-1),1} \times n_{(\ll-1),2} \times \ldots n_{(\ll-1),N }, & \mathbf{n_{(\ll-1)}} ) \\
|| \y_\llp || &  = & (n_{(\ll-1),1} \times n_{(\ll-1),2} \times \ldots n_{(\ll-1),N},  &\mathbf{n_\llp} )
\end{array}
$$




# Conv2d: Two dimensional convolution ($N = 2$)

Thus far, the spatial dimension has been of length $N = 1$.

Generalizing  to $N = 2$ is straightforward.
- The number of spatial dimensions (elements denoted by $\ldots$) expands from $1$ to $2$

<div>
    <br>
    <center><strong>Conv 2D: single input feature: kernel 1</strong></center>
    <br>
<img src=images/Conv2d_singlefeature_input_kernel.png width="35%">
    <!-- edX: obsolete: W9_L2_S37_Conv2d_singlefeature_input_kernel.png EdX created image --> 
    <br>
    </div>
    
- $\mathbf{W}_{\llp, j', \ldots,j}$
    - layer $\ll$
    - output feature $j$
    - spatial location: $\ldots \in \{ ( \alpha, \alpha' ) \in (\dim_{(\ll-1),1} \times \dim_{(\ll-1),2} \}$
    - input feature $j'$    

Here is a [picture](CNN_pictorial.ipynb#Conv-2D:-single-feature-to-single-feature) of a Convolutional layer $\ll$
transforming 
- a  2-dimensional input layer $(\ll-1)$  consisting of a 1 feature 
    - $N_{(\ll-1)} = 2, n_{(\ll-1)} = 1$
- into a 2-dimensional output layer $\ll$  consisting of 1 feature
    - $N_\llp = 1, n_\llp  = 1$



We can further generalize to producing multiple output features

Here is a [picture](CNN_pictorial.ipynb#Conv-2D:-single-feature-to-multiple-features) of a Convolutional layer $\ll$
transforming 
- a  2-dimensional input layer $(\ll-1)$  consisting of a 1 feature 
    - $N_{(\ll-1)} = 2, n_{(\ll-1)} = 1$
- into a 2-dimensional output layer $\ll$  consisting of 2 feature
    - $N_\llp = 1, n_\llp  = 2$


Dealing with multiple input features works similarly as for $N=1$:
- The dot product
- Is over a spatial region that now has a "depth" $n_{(\ll-1)}$ equal to the number of input features
- Which means the kernel has a depth $n_{(\ll-1)}$

<div>
    <br>
    <center><strong>Conv 2D: multiple input features: kernel 1</strong></center>
    <br>
<img src=images/Conv2d_multifeature_input_kernel.png width="50%">
    <!-- edX: obsolete W9_L2_S46_Conv2d_multifeature_input_kernel.png EdX created image --> 
    <br>
    </div>

Here is a [picture](CNN_pictorial.ipynb#Conv-2D:-multiple-features-to-single-feature) of a Convolutional layer $\ll$
transforming 
- a  2-dimensional input layer $(\ll-1)$  consisting of multiple features 
    - $N_{(\ll-1)} = 2, n_{(\ll-1)} = 2$
- into a 2-dimensional output layer $\ll$  consisting of 1 feature
    - $N_\llp = 1, n_\llp  = 1$


And finally: the general case for a 2 spatial dimensions
    
Here is a [picture](CNN_pictorial.ipynb#Conv-2D:-multiple-features-to-multiple-features) of a Convolutional layer $\ll$
transforming 
- a  2-dimensional input layer $(\ll-1)$  consisting of multiple features 
    - $N_{(\ll-1)} = 2, n_{(\ll-1)} = 3$
- into a 2-dimensional output layer $\ll$  consisting of multiple features
    - $N_\llp = 1, n_\llp  = 2$



# Training a CNN

Hopefully you understand how kernels are "feature recognizers".

But you may be wondering: how do we determine the weights in each kernel ?

Answer: a Convolutional Layer is "just another" layer in a multi-layer network
- The kernels are just weights (like the weights in Fully Connected layers)
- We solve for all the weights $\W$ in the multi-layer network in the same way


The answer is: exactly as we did in Classical Machine Learning
- Define a loss function that is parameterized by $\W$: 
$$\loss = L(\hat{\y},\y; \W)$$

- The kernel weights are just part of $\W$
- Our goal is to find $\W^*$ the "best" set of weights
$$
\W^* = \argmin{W} L(\hat{\y},\y; \W)
$$
- Using Gradient Descent !

In other words: their is nothing special about finding the "best" kernels.

In [5]:
print("Done")

Done
